In [1]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('nbagg')
import matplotlib.animation as anm
import math
import matplotlib.patches as patches
import numpy as np
from IPython.display import HTML

In [9]:
class World:
    def __init__(self,debug=False):
        self.objects = []
        self.debug = debug
        
    def append(self, obj):
        self.objects.append(obj)
        
    def draw(self):
        fig = plt.figure(figsize=(4,4))
        ax = fig.add_subplot(111)
        ax.set_aspect('equal')
        ax.set_xlim(-5,5)
        ax.set_ylim(-5,5)
        ax.set_xlabel("X",fontsize=10)
        ax.set_ylabel("Y",fontsize=10)
        
        elems = []
        
        if self.debug:
            for i in range(1000): self.one_step(i,elems,ax)
        else:
            self.ani = anm.FuncAnimation(fig, self.one_step, fargs=(elems,ax),frames=10,interval=1000,repeat=False)
            #plt.show()
            plt.close()
            return HTML(self.ani.to_jshtml())
        
        # for obj in self.objects: obj.draw(ax)
        # plt.show()
    
    def one_step(self, i, elems, ax):
        while elems: elems.pop().remove()
        elems.append(ax.text(-4.4,4.5,"t="+str(i),fontsize=10))
        for obj in self.objects:
            obj.draw(ax,elems)

In [12]:
class IdealRobot:
    def __init__(self, pose, color="black"):
        self.pose = pose
        self.r = 0.2
        self.color = color
        
    def draw(self, ax, elems):
        x,y,theta = self.pose
        xn = x + self.r * math.cos(theta)
        yn = y + self.r * math.sin(theta)
        elems += ax.plot([x,xn],[y,yn],color=self.color)
        c = patches.Circle(xy=(x,y), radius=self.r, fill=False, color=self.color)
        elems.append(ax.add_patch(c))
        
    @classmethod
    def state_transition(cls, nu, omega, time, pose):
        t0 = pose[2]
        if math.fabs(omega) < 1e-10:
            return pose + np.array( [nu*math.cos(t0), 
                                     nu*math.sin(t0),
                                     omega ] ) * time
        else:
            return pose + np.array( [nu/omega*(math.sin(t0 + omega*time) - math.sin(t0)), 
                                     nu/omega*(-math.cos(t0 + omega*time) + math.cos(t0)),
                                     omega*time ] )

In [14]:
IdealRobot.state_transition(0.1,30.0/180.0*math.pi,1.0,np.array([1,0,0]).T)

array([1.09549297, 0.02558726, 0.52359878])

In [11]:
world = World()
robot1 = IdealRobot(np.array([2,3,math.pi/6]).T)
robot2 = IdealRobot(np.array([-1,-3,math.pi*5/6]).T,"red")
world.append(robot1)
world.append(robot2)
world.draw()